In [4]:
#import torch
import open3d.ml.torch as ml3d

In [8]:
#Read a dataset by specifying the path. We are also providing the cache directory and training split.
dataset = ml3d.datasets.SemanticKITTI(dataset_path='/Users/sanskara/data/SemanticKITTI/', cache_dir='./logs/cache',training_split=['00'], validation_split=['01'], test_split=['01'])

In [7]:
#Split the dataset for 'training'. You can get the other splits by passing 'validation' or 'test'
train_split = dataset.get_split('training')

INFO - 2022-03-24 19:48:01,057 - semantickitti - Found 1 pointclouds for training


In [10]:
#Similarly, get validataion split.
val_split = dataset.get_split('validation')

INFO - 2022-03-24 19:49:34,283 - semantickitti - Found 1 pointclouds for validation


In [6]:
#get test split
test_split = dataset.get_split('test')

INFO - 2022-03-24 19:47:58,953 - semantickitti - Found 1 pointclouds for test


In [13]:
# Get length of splits
print(len(train_split))
print(len(val_split))
print(len(test_split))

1
1
1


In [18]:
# Query splits for data, index should be from `0` to `len(split) - 1`
for i in range(len(train_split)):
    data = train_split.get_data(i)
    print(data)
    break

{'point': array([[ 5.2305943e+01,  2.2989707e-02,  1.9779946e+00],
       [ 5.3259735e+01,  1.0695236e-01,  2.0099745e+00],
       [ 5.3284321e+01,  2.7487758e-01,  2.0109341e+00],
       ...,
       [ 3.8249431e+00, -1.4261885e+00, -1.7655631e+00],
       [ 3.8495324e+00, -1.4222100e+00, -1.7755738e+00],
       [ 3.8631279e+00, -1.4142324e+00, -1.7805853e+00]], dtype=float32), 'feat': None, 'label': array([0, 0, 0, ..., 9, 9, 9], dtype=int32)}


In [24]:
data = train_split.get_data(0) # Dictionary of `point`, `feat`, and `label`
print(data.keys())

dict_keys(['point', 'feat', 'label'])


In [23]:
attr = train_split.get_attr(0)
print(attr)  # Dictionary containing information about the data e.g. name, path, split, etc.

{'idx': 0, 'name': '00_000001', 'path': '/Users/sanskara/data/SemanticKITTI/dataset/sequences/00/velodyne/000001.bin', 'split': 'training'}


In [ ]:
#support of Open3d-ML visualizer in Jupyter Notebooks is in progress
#view the frames using the visualizer
#vis = ml3d.vis.Visualizer()
#vis.visualize_dataset(dataset, 'training',indices=range(len(train_split)))